# Crypto Data Fetcher

This Jupyter Notebook is designed to fetch and save cryptocurrency data using the Coinlore API. The notebook includes functions to:

1. Retrieve cryptocurrency data from the Coinlore API.
2. Save the retrieved data to a JSON file with a timestamped filename.
3. Interactively input the starting rank and the number of cryptocurrencies to fetch.

The main function orchestrates the workflow by prompting the user for input, fetching the data, and saving it to a file.

Here is the link to the Coinlore API documentation: https://www.coinlore.com/cryptocurrency-data-api

In [2]:
import requests
import datetime
import pandas as pd
from datetime import datetime
import pprint as pp
import json

In [3]:
url = f"https://api.coinlore.net/api/tickers/"
response = requests.get(url)
raw_json = response.json()
pp.pprint(raw_json)

{'data': [{'csupply': '19831334.00',
           'id': '90',
           'market_cap_usd': '1687617555194.20',
           'msupply': '21000000',
           'name': 'Bitcoin',
           'nameid': 'bitcoin',
           'percent_change_1h': '-1.13',
           'percent_change_24h': '1.87',
           'percent_change_7d': '1.93',
           'price_btc': '1.00',
           'price_usd': '85098.54',
           'rank': 1,
           'symbol': 'BTC',
           'tsupply': '19831334',
           'volume24': 33069028833.51779,
           'volume24a': 23201533761.432915},
          {'csupply': '120473321.00',
           'id': '80',
           'market_cap_usd': '238305175338.05',
           'msupply': '',
           'name': 'Ethereum',
           'nameid': 'ethereum',
           'percent_change_1h': '-1.64',
           'percent_change_24h': '1.70',
           'percent_change_7d': '3.89',
           'price_btc': '0.023244',
           'price_usd': '1978.07',
           'rank': 2,
           'symbol':

In [4]:
timestamp = datetime.fromtimestamp(raw_json['info']['time']).strftime('%Y-%m-%d %H:%M:%S')
print(timestamp)

2025-03-20 11:28:04


In [5]:
df = pd.DataFrame(raw_json['data'])

#drop the rank column
df = df.drop(columns=['rank'])

#add the timestamp to the dataframe
df['timestamp'] = timestamp

print(df.head())
print("\n\n")

#replace all the '' values with None
df = df.replace('', None)

     id symbol          name        nameid price_usd percent_change_24h  \
0    90    BTC       Bitcoin       bitcoin  85098.54               1.87   
1    80    ETH      Ethereum      ethereum   1978.07               1.70   
2    58    XRP           XRP        ripple      2.48               7.18   
3   518   USDT        Tether        tether      1.00              -0.12   
4  2710    BNB  Binance Coin  binance-coin    629.85               2.56   

  percent_change_1h percent_change_7d price_btc    market_cap_usd  \
0             -1.13              1.93      1.00  1687617555194.20   
1             -1.64              3.89  0.023244   238305175338.05   
2             -0.55             11.12  0.000029   143078613260.48   
3              0.09              0.12  0.000012   139576729917.41   
4             -0.95              9.67  0.007401   105059782497.08   

       volume24     volume24a          csupply          tsupply       msupply  \
0  3.306903e+10  2.320153e+10      19831334.00       

In [6]:
df['price_usd'] = df['price_usd'].astype(float)
df['percent_change_24h'] = df['percent_change_24h'].astype(float)
df['percent_change_1h'] = df['percent_change_1h'].astype(float)
df['percent_change_7d'] = df['percent_change_7d'].astype(float)
df['price_btc'] = df['price_btc'].astype(float)
df['market_cap_usd'] = df['market_cap_usd'].astype(float)
df['volume24'] = df['volume24'].astype(float)
df['volume24a'] = df['volume24a'].astype(float)
df['csupply'] = df['csupply'].astype(float)
df['tsupply'] = df['tsupply'].astype(float)
df['msupply'] = df['msupply'].astype(float) 

print(df.head())

     id symbol          name        nameid  price_usd  percent_change_24h  \
0    90    BTC       Bitcoin       bitcoin   85098.54                1.87   
1    80    ETH      Ethereum      ethereum    1978.07                1.70   
2    58    XRP           XRP        ripple       2.48                7.18   
3   518   USDT        Tether        tether       1.00               -0.12   
4  2710    BNB  Binance Coin  binance-coin     629.85                2.56   

   percent_change_1h  percent_change_7d  price_btc  market_cap_usd  \
0              -1.13               1.93   1.000000    1.687618e+12   
1              -1.64               3.89   0.023244    2.383052e+11   
2              -0.55              11.12   0.000029    1.430786e+11   
3               0.09               0.12   0.000012    1.395767e+11   
4              -0.95               9.67   0.007401    1.050598e+11   

       volume24     volume24a       csupply       tsupply       msupply  \
0  3.306903e+10  2.320153e+10  1.983133e+

In [7]:
df.to_json('data.json')
df.to_parquet('data.parquet')

Traitement full JSON

In [16]:
raw_json = response.json()

#add timestamp key and value to every object in data array
timestamp = datetime.fromtimestamp(raw_json['info']['time']).strftime('%Y-%m-%d %H:%M:%S')

for i in range(len(raw_json['data'])):
    raw_json['data'][i]['timestamp'] = timestamp

#drop the info key
raw_json = raw_json['data']

#drop the rank key from every object in array
for i in range(len(raw_json)):
    del raw_json[i]['rank']

#replace all the '' values with None
for i in range(len(raw_json)):
    for key in raw_json[i]:
        if raw_json[i][key] == '':
            raw_json[i][key] = None

#list all keys that need to be converted to float
keys_to_float = ['price_usd', 'percent_change_24h', 'percent_change_1h', 'percent_change_7d', 'price_btc', 'market_cap_usd', 'volume24', 'volume24a', 'csupply', 'tsupply', 'msupply']

#convert all the values to the correct data type if not None
for i in range(len(raw_json)):
    for key in raw_json[i]:
        if raw_json[i][key] != None and key in keys_to_float:
                raw_json[i][key] = float(raw_json[i][key])


#write the json to a file
with open('data2.json', 'w') as f:
    json.dump(raw_json, f)

In [17]:
#write the json to a parquet file
df = pd.DataFrame(raw_json)
df.to_parquet('data2.parquet')